<h>Test of subarray power on sequence.</h>

First defines all elements in chart. Configuration uses one subarray, two subarray beams, four apertures in two station. 
This should be parametrizable

In [141]:
import tango
import time
import json
import numpy as np

from ska_control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    ResultCode,
    TaskStatus,
    SimulationMode,
    TestMode,
)

# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"
# define devices
controller = tango.DeviceProxy('low-mccs/control/control')
sa1 = tango.DeviceProxy('low-mccs/subarray/01')
sa2 = tango.DeviceProxy('low-mccs/subarray/02')
sab1 = tango.DeviceProxy('low-mccs/subarraybeam/01')
sab2 = tango.DeviceProxy('low-mccs/subarraybeam/02')
st1 = tango.DeviceProxy('low-mccs/station/001')
st2 = tango.DeviceProxy('low-mccs/station/002')
sb1 = tango.DeviceProxy('low-mccs/beam/001')
sb2 = tango.DeviceProxy('low-mccs/beam/002')
sb3 = tango.DeviceProxy('low-mccs/beam/003')
sb4 = tango.DeviceProxy('low-mccs/beam/004')
spsst1 = tango.DeviceProxy('low-mccs/spsstation/ci-1')
spsst2 = tango.DeviceProxy('low-mccs/spsstation/ci-2')
sr1 = tango.DeviceProxy('low-mccs/subrack/ci-1-1')
sr2 = tango.DeviceProxy('low-mccs/subrack/ci-2-1')
t1 = tango.DeviceProxy('low-mccs/tile/ci-1-13')
t2 = tango.DeviceProxy('low-mccs/tile/ci-1-17')
t3 = tango.DeviceProxy('low-mccs/tile/ci-2-13')
t4 = tango.DeviceProxy('low-mccs/tile/ci-2-17')
cs1 = tango.DeviceProxy('low-mccs/calibrationstore/ci-1')
mfs1 = tango.DeviceProxy('low-mccs/mockfieldstation/ci-1')
mfs2 = tango.DeviceProxy('low-mccs/mockfieldstation/ci-2')
stc1 = tango.DeviceProxy('low-mccs/stationcalibrator/ci-1')
stc2 = tango.DeviceProxy('low-mccs/stationcalibrator/ci-2')
devices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4, st1, st2, 
           sb1, sb2, sb3, sb4, sab1, sab2, sa1, sa2,
          mfs1, mfs2, stc1, stc2, cs1]
hwdevices = [spsst1, spsst2, sr1, sr2, t1, t2, t3, t4]
sps_stations = [spsst1, spsst2]

Put everything online and ON. Logging level for some devices is raised to DEBUG.  

In [142]:
for d in devices:
    d.adminmode = 0
    d.logginglevel = 5
controller.adminmode=0
controller.logginglevel = 5
sa1.logginglevel = 5
sab1.logginglevel = 5
sb1.logginglevel = 5


Set station parameters. Site dependent

In [ ]:
for station in sps_stations:
    station.SetBeamformerTable([128,0,1,0,0,0,0])
    station.statictimedelays=np.zeros([512],dtype=int)
    station.preaduLevels=list(range(32))*16
    station.channeliserRounding=[4]*512
    station.cspRounding=[4]*384
    station.SetLmcDownload('{"destination_ip": "10.0.0.98", "mode": "10g"}')
    station.SetLmcIntegratedDownload('{"destination_ip": "10.0.0.98", "mode": "10g"}')
    station.SetCspIngest('{"destination_ip": "10.0.0.98"}')

Initialise all tiles. 
If all tiles are already synchronised, assume that they are properly so.
If not: 
- power on and initialise tiles which are off
- initialise tiles which are on
- perform station initialisation
- reinitialise stations which were not completely on or off (should be rare)
- wait for initialisation to complete
- synchronise all stations (together)

In [143]:
# Executes initialization if required. 
# If all tiles are synchronized, it is assumed they are properly so
#
if any(any(
        state != 'Synchronised' for state in station.tileProgrammingState)
        for station in sps_stations):   
    print("Must turn on and reinitialise stations")
    reinitialize = False  # second  initialization is required if not all tiles 
                          # are initialised in first cycle
    # Turn on tiles which are off and initialises tiles whch are off. 
    for station in sps_stations:
        if station.state() == tango.DevState.ON:
            all_off = False
            station.Initialise() # re-initialise if already ON
        elif station.state() in (tango.DevState.OFF, tango.DevState.STANDBY):
            station.on() # ON includes reinitialisation
        else:
            station.on() # turn on what is not already on, but must reinitialze
            reinitialize = True
    timeout = 60 # seconds
    print("Waiting for all stations to come ON and initialise")
    while timeout > 0:
        time.sleep(2)
        if all(
            all(
               status in ('Initialised', 'Synchronized')
               for status in station.tileProgrammingState)
               for station in sps_stations):
            break
        timeout -= 2
    if timeout <= 0:
        print("Error: timeout in waiting for stations to initialize")         
    else:
        time.sleep(2)   # to wait for station intialisation to complete 
        if reinitialize:
            # re-initialisation required. Some stations were not ON or OFF
            for station in sps_stations:
                station.initialise()
                timeout = 60 # seconds
                print("Waiting for all remaining unprogrammed tiles initialise")
                while timeout > 0:
                    time.sleep(2)
                    if all(all(
                           status == 'Initialised' for status in station.tileProgrammingState)
                           for station in sps_stations):
                        break
                if timeout <= 0: 
                    print("Error: timeout in waiting for tiles to initialize") 
        #
        # Synchronize all stations
        #
        time.sleep(2)
        print("Synchronize all stations")
        start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
        for station in sps_stations:
            station.StartAcquisition(json.dumps({"start_time": start_time}))
        #
        # check that synchronization worked
        #
        time.sleep(3)
#
# Check for successful synchronisation
if any(
        any(state !='Synchronised' for state in station.tileprogrammingstate) 
        for station in sps_stations):
    print("stations not synchronised")
    for station in sps_stations:
        print(f'{station.name()}: state = {station.tileprogrammingstate}')
else:
    print("MCCS system initialised")

MCCS system initialised
